## Кэширование данных для ускорения запросов

Задача: реализовать кэширование результата длительного SQL-запроса или API-вызова в Redis.

Цель: после первого запроса данные сохраняются в Redis, последующие обращения загружают их из кэша, значительно снижая время отклика.

Используемые команды: SET, GET, EXPIRE для автоматического удаления устаревших данных

In [3]:
import hashlib
import json
import time
import redis
from clickhouse_driver import Client

In [4]:
r = redis.Redis(
    host='localhost',
    port=6379,
    db=0,
    decode_responses=True
)

In [5]:
ch = Client(
    host='localhost',
    port=9000,
    user='admin',
    password='admin',
    database='test'
)

In [6]:
ch.execute("DROP TABLE IF EXISTS user")
ch.execute(
    """
    CREATE TABLE IF NOT EXISTS user (
    id UInt32,
    name String,
    age UInt8,
    signup_data DateTime DEFAULT now()
    ) ENGINE MergeTree()
    ORDER BY id;
    """
)

[]

In [7]:
ch.execute(
    """
    INSERT INTO user (id, name, age, signup_data) 
    VALUES (1, 'John', 32, now()), (2, 'Judie', 25, now());
    """
)

[]

In [8]:
def generate_cache_key(query):
    return "ca:" + hashlib.md5(query.encode("utf-8")).hexdigest()


def fetch_data_from_db(query):
    data = ch.execute(query)
    return json.dumps(data, default=str)


def get_data_with_cache(query, ttl = 300):
    cache_key = generate_cache_key(query)
    cached_data = r.get(cache_key)

    if cached_data is not None:
        print("cache get")
        return json.loads(cached_data)
    else:
        print("data get")
        time.sleep(10)
        data = fetch_data_from_db(query)
        data_json = json.dumps(data, default=str) 
        r.setex(cache_key, ttl, data_json)
        return data

In [12]:
sql_query = "SELECT * FROM user"
result = get_data_with_cache(sql_query)
print(result)

cache get
[[1, "John", 32, "2025-08-22 07:41:35"], [2, "Judie", 25, "2025-08-22 07:41:35"]]


In [10]:
sql_query = "SELECT * FROM user"
res = ch.execute(sql_query)
print(res)

[(1, 'John', 32, datetime.datetime(2025, 8, 22, 7, 41, 35)), (2, 'Judie', 25, datetime.datetime(2025, 8, 22, 7, 41, 35))]
